<a href="https://colab.research.google.com/github/artinmajdi/chest_xray_private_main/blob/master/aims/aim1_1_taxonomy/SAM/my_SAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook how to minimally implement **sharpness-aware minimization** in TensorFlow with the CIFAR10 dataset. Sharpness-aware minimization (SAM) was proposed in the paper - [Sharpness-Aware Minimization for Efficiently Improving Generalization](https://openreview.net/pdf?id=6Tm1mposlrM)<sup>*</sup>. Some notable differences in this implementation: 
* ResNet20 (attributed to [this repository](https://github.com/GoogleCloudPlatform/keras-idiomatic-programmer/blob/master/zoo/resnet/resnet_cifar10.py)) is used as opposed to PyramidNet and WideResNet. 
* ShakeDrop regularization has not been used.
* Two simple augmentation transformations (random crop and random brightness) have been used as opposed to Cutout, AutoAugment. 
* Adam has been used as the optimizer with the default arguments as provided by TensorFlow with a `ReduceLROnPlateau`. Table 1 of the original paper suggests to use SGD with different configurations. 
* Instead of training for full number of epochs I used early stopping with a patience of 10.

I referred to the following resources for this study - 
* [Original Paper](https://openreview.net/pdf?id=6Tm1mposlrM) 
* [davda54](https://github.com/davda54)'s [PyTorch implementation](https://github.com/davda54/sam)

*<sub>arXiv version of the paper can be found [here](https://arxiv.org/abs/2010.01412).</sub>

## Initial Setup

In [13]:
from google.colab import drive
drive.mount('/content/drive')

# !git clone https://github.com/sayakpaul/Sharpness-Aware-Minimization-TensorFlow

Mounted at /content/drive


In [2]:
!pip install -r /content/drive/MyDrive/RESEARCH/PhD/code/my_main_code/my_main_code/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 64.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 28.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.4/134.4 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 80.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0

In [3]:
# !pip install -e '/content/drive/MyDrive/RESEARCH/PhD/code/my_main_code/'

import my_main_code

In [4]:
# !pip install condacolab
# import condacolab
# condacolab.install()
# !conda install git gitpython

In [12]:
# !pip install colabcode
# from colabcode import ColabCode
# ColabCode(port=1000)

In [18]:
%reload_ext autoreload
%autoreload 2

import sys
import os

# dir = '/content/drive/MyDrive/RESEARCH/PhD/code/my_main_code/'
# sys.path.append(dir)

import time
import tensorflow as tf
tf.random.set_seed(42)
print('tf version:', tf.__version__)


from my_main_code.utils import funcs
from my_main_code.aims.aim1_1_taxonomy.SAM import utils 
from my_main_code.aims.aim1_1_taxonomy.SAM import resnet_cifar10
# from aims.aim1_1_taxonomy.SAM import utils, resnet_cifar10


%reload_ext my_main_code.aims.aim1_1_taxonomy.SAM.resnet_cifar10
%reload_ext my_main_code.aims.aim1_1_taxonomy.SAM.utils
%reload_ext my_main_code.utils.funcs

tf version: 2.9.2


In [6]:
artin_stuff = utils.ArtinStuff()

strategy = artin_stuff.tpu_gpu_initialization()

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.10.154.186:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.10.154.186:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


## Load Dataset and Prepare Data Loaders

### <span style='color:red'> a. Original dataset loading code from Google/SAM </span>

In [24]:
X , Y = artin_stuff.load_cifar100_raw_data()

mode='merged' # 'fine' 'coarse' 'merged'
n_classes = 120 # Y[mode]['train'].shape[1]
X[mode]['test'].shape

(10000, 32, 32, 3)

In [25]:
train_ds, test_ds = artin_stuff.load_cifar100_dataset( X=X, Y=Y, strategy=strategy,  mode=mode,  batch_size=128 )

Batch size: 1024


## Encapsulate SAM Logic 

SAM is implemented as follows - 

<center>
<img src="https://i.ibb.co/qRSfNX7/image.png"></img><br>
<small>Source: Original Paper</small>
</center>

## Define Callbacks

## Initialize Model with SAM and Train It

In [ ]:
# history = artin_stuff.fit_SAM(strategy=strategy ,  train_ds=train_ds ,  test_ds=test_ds, n_classes=n_classes, activation='softmax', loss='sparse_categorical_crossentropy')

# utils.plot_history(history)

## Train a Regular ResNet20 Model

In [28]:
# num_classes = 120 if mode=='merged' else 100
model = funcs.architecture(architecture_name='EfficientNetB3', input_shape=[224,224,3], num_classes=n_classes, activation='softmax', first_index_trainable=-5)

model.compile(  optimizer="adam", loss=tf.keras.losses.binary_crossentropy, metrics=["accuracy"] ) 

In [29]:
start = time.time()

history = model.fit(train_ds,  validation_data=test_ds,  callbacks=artin_stuff.train_callbacks(),  epochs=200)
print(f"Total training time: {(time.time() - start)/60.} minutes")

utils.plot_history(history)

Epoch 1/200
49/49 [==============================] - 551s 11s/step - loss: 0.1437 - accuracy: 0.0429 - val_loss: 1.5131 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 2/200
49/49 [==============================] - 542s 11s/step - loss: 0.0801 - accuracy: 0.0500 - val_loss: 1.3020 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 3/200
49/49 [==============================] - 541s 11s/step - loss: 0.0800 - accuracy: 0.0514 - val_loss: 0.8262 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 4/200
49/49 [==============================] - 546s 11s/step - loss: 0.0799 - accuracy: 0.0524 - val_loss: 0.4268 - val_accuracy: 0.0500 - lr: 0.0010
Epoch 5/200
49/49 [==============================] - 543s 11s/step - loss: 0.0799 - accuracy: 0.0519 - val_loss: 0.2236 - val_accuracy: 0.0500 - lr: 0.0010
Epoch 6/200
49/49 [==============================] - 545s 11s/step - loss: 0.0799 - accuracy: 0.0528 - val_loss: 0.1392 - val_accuracy: 0.0500 - lr: 0.0010
Epoch 7/200
49/49 [==============================] -

KeyboardInterrupt: ignored